## Librerias

In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
from PIL import Image
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer,normalize,RobustScaler,MaxAbsScaler,QuantileTransformer
import warnings
from sklearn.metrics import root_mean_squared_error
warnings.filterwarnings('ignore')

## Data

In [79]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv("Data/train_clean.csv")


## Data exploration

In [80]:
train.head()

,laptop_ID,TypeName,Inches,Touchscreen,ProcessorType,ModelIdentifier,Core,ProcessorSpeed(GHz),Ram(GB),Memory SSD GB,...,"PanelType_IPS Panel, Quad HD+","PanelType_IPS Panel, Retina Display",PanelType_No penal type,PanelType_Quad HD+,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS,OpSys_No OS,OpSys_Windows
0,599,1,11.6,1,9.0,730,0.0,1.0,4,128.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,92,2,15.6,0,10.0,9830,0.0,3.0,8,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,744,3,15.6,0,1.0,7700,0.0,2.8,16,256.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,729,1,13.3,1,5.0,7200,0.0,2.5,8,256.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,957,4,15.6,0,5.0,6200,0.0,2.3,8,256.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Data processing

In [81]:
df = train.copy()

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 55 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   laptop_ID                            912 non-null    int64  
 1   TypeName                             912 non-null    int64  
 2   Inches                               912 non-null    float64
 3   Touchscreen                          912 non-null    int64  
 4   ProcessorType                        911 non-null    float64
 5   ModelIdentifier                      912 non-null    int64  
 6   Core                                 912 non-null    float64
 7   ProcessorSpeed(GHz)                  912 non-null    float64
 8   Ram(GB)                              912 non-null    int64  
 9   Memory SSD GB                        912 non-null    float64
 10  Memory HDD GB                        912 non-null    float64
 11  Memory FL STO GB                

In [83]:
df['ProcessorType'] = df['ProcessorType'].fillna(15)

-----------------------------------------------------------------------------------------------------------------

In [84]:
# corr_matrix = df.corr()
# # Heatmap matriz de correlaciones
# # ==============================================================================
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(55, 30))

# sns.heatmap(
#     corr_matrix,
#     annot     = True,
#     cbar      = False,
#     annot_kws = {"size": 8},
#     vmin      = -1,
#     vmax      = 1,
#     center    = 0,
#     # cmap      = sns.diverging_palette(20, 220, n=200),
#     square    = True,
#     ax        = ax
# )

# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation = 45,
#     horizontalalignment = 'right',
# )

# ax.tick_params(labelsize = 10)

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [85]:
import numpy as np

In [86]:
# variables_insignificantes = ['Ancho','Alto','AspectRatio','Inches','OpSys','Weight(KG)']

# # Eliminar estas variables del modelo
# df = df.drop(columns=variables_insignificantes)

In [87]:
# X = df.drop(['Price_euros'], axis=1)
# y = df['Price_euros'].copy()
X = df.drop(['Price_euros'], axis=1)
y = df['Price_euros'].values.reshape(-1, 1)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)
X_scaled = scaler_X.fit_transform(X)

### 2. Dividir X_train, X_test, y_train, y_test

In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size = 0.20, random_state = 42)

In [89]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calcular el VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

# Mostrar variables con alto VIF
print(vif_data[vif_data["VIF"] > 10])  # Si VIF > 10, eliminar esa variable

                               Variable           VIF
16                                Ancho  2.640843e+04
17                                 Alto  3.282391e+04
18                              Pixeles  1.883730e+03
19                          AspectRatio  3.587743e+02
20                         Company_Acer           inf
21                        Company_Apple           inf
22                         Company_Asus           inf
23                        Company_Chuwi           inf
24                         Company_Dell           inf
25                      Company_Fujitsu           inf
26                       Company_Google           inf
27                           Company_HP           inf
28                       Company_Huawei           inf
29                           Company_LG           inf
30                       Company_Lenovo           inf
31                          Company_MSI           inf
32                     Company_Mediacom           inf
33                    Compan

In [90]:
import statsmodels.api as sm

# Crear modelo de regresión
X_const = sm.add_constant(X_scaled)  # Agregar constante
modelo = sm.OLS(y_scaled, X_const).fit()

# Mostrar el resumen con p-values
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     58.97
Date:                Sun, 02 Feb 2025   Prob (F-statistic):          1.05e-241
Time:                        18:01:45   Log-Likelihood:                 1258.7
No. Observations:                 912   AIC:                            -2415.
Df Residuals:                     861   BIC:                            -2170.
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3893      0.208     -1.868      0.0

### 3. Asignar el modelo (vacío) a una variable


In [91]:
# from sklearn.linear_model import LinearRegression

# regressor = LinearRegression()

# regressor.fit(X_train, y_train)

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de RMSE.

In [92]:
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train, y_train.ravel())

# Hacer predicciones
y_pred_rf = rf.predict(X_test)

# Calcular RMSE
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)
print(f"✅ RMSE Random Forest: {rmse_rf:.4f}")

✅ RMSE Random Forest: 0.0453


In [93]:
# predictions = regressor.predict(X_test)

# rmse = root_mean_squared_error(y_test, predictions)
# rmse

In [94]:
# X_test.columns

### 2. Carga los datos de `test.csv` para predecir.

In [95]:
X_pred = pd.read_csv("Data/test_clean.csv")
# X_pred = X_pred.drop(columns=variables_insignificantes)
X_pred.columns

Index(['laptop_ID', 'TypeName', 'Inches', 'Touchscreen', 'ProcessorType',
       'ModelIdentifier', 'Core', 'ProcessorSpeed(GHz)', 'Ram(GB)',
       'Memory SSD GB', 'Memory HDD GB', 'Memory FL STO GB',
       'Memory Hybrid GB', 'GpuBrand', 'GpuModel', 'Weight(KG)', 'Ancho',
       'Alto', 'Pixeles', 'AspectRatio', 'Company_Acer', 'Company_Apple',
       'Company_Asus', 'Company_Dell', 'Company_Fujitsu', 'Company_HP',
       'Company_LG', 'Company_Lenovo', 'Company_MSI', 'Company_Mediacom',
       'Company_Microsoft', 'Company_Razer', 'Company_Samsung',
       'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi',
       'PanelType_4K Ultra HD', 'PanelType_Full HD', 'PanelType_IPS Panel',
       'PanelType_IPS Panel, 4K Ultra HD', 'PanelType_IPS Panel, Full HD',
       'PanelType_IPS Panel, Quad HD+', 'PanelType_IPS Panel, Retina Display',
       'PanelType_No penal type', 'PanelType_Quad HD+', 'OpSys_Android',
       'OpSys_Chrome OS', 'OpSys_Linux', 'OpSys_Mac OS', 'OpSys_No OS',
    

In [96]:
X_pred = X_pred[['laptop_ID', 'TypeName', 'Inches', 'Touchscreen', 'ProcessorType',
       'ModelIdentifier', 'Core', 'ProcessorSpeed(GHz)', 'Ram(GB)',
       'Memory SSD GB', 'Memory HDD GB', 'Memory FL STO GB',
       'Memory Hybrid GB', 'GpuBrand', 'GpuModel', 'Weight(KG)',
       'Ancho', 'Alto', 'Pixeles', 'AspectRatio', 'Company_Acer',
       'Company_Apple', 'Company_Asus', 'Company_Chuwi', 'Company_Dell',
       'Company_Fujitsu', 'Company_Google', 'Company_HP', 'Company_Huawei',
       'Company_LG', 'Company_Lenovo', 'Company_MSI', 'Company_Mediacom',
       'Company_Microsoft', 'Company_Razer', 'Company_Samsung',
       'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi',
       'PanelType_4K Ultra HD', 'PanelType_Full HD', 'PanelType_IPS Panel',
       'PanelType_IPS Panel, 4K Ultra HD', 'PanelType_IPS Panel, Full HD',
       'PanelType_IPS Panel, Quad HD+', 'PanelType_IPS Panel, Retina Display',
       'PanelType_No penal type', 'PanelType_Quad HD+', 'OpSys_Android',
       'OpSys_Chrome OS', 'OpSys_Linux', 'OpSys_Mac OS', 'OpSys_No OS',
       'OpSys_Windows']]

# X_pred = X_pred[['laptop_ID', 'TypeName', 'Inches', 'PanelType', 'Touchscreen',
#        'ProcessorType', 'ModelIdentifier', 'Core', 'ProcessorSpeed(GHz)',
#        'Ram(GB)', 'Memory SSD GB', 'Memory HDD GB', 'Memory FL STO GB',
#        'Memory Hybrid GB', 'GpuBrand', 'GpuModel', 'OpSys', 'Weight(KG)', 'Ancho', 'Alto', 'Pixeles', 'AspectRatio',
#        'Company_Acer', 'Company_Apple', 'Company_Asus', 'Company_Dell',
#        'Company_HP', 'Company_Lenovo', 'Company_MSI', 'Company_Otros',
#        'Company_Toshiba']]

In [97]:
# 1️⃣ Transformar X_pred con el escalador de X
X_pred_scaled = scaler_X.transform(X_pred)

# 2️⃣ Hacer predicciones con el modelo entrenado
predictions_scaled = rf.predict(X_pred_scaled)

# 3️⃣ Asegurar que las predicciones tienen la forma correcta (matriz 2D)
predictions_scaled = predictions_scaled.reshape(-1, 1)

# 4️⃣ Desnormalizar las predicciones con scaler_y
predictions_submit = scaler_y.inverse_transform(predictions_scaled)

In [98]:
predictions_submit

array([[1266.52627817],
       [1161.99132051],
       [ 264.01111349],
       [1941.6968871 ],
       [1616.02647498],
       [ 389.26513264],
       [2565.90459286],
       [ 468.26065937],
       [1765.87609803],
       [ 784.10407511],
       [1098.6106665 ],
       [2606.73521158],
       [ 513.22079397],
       [ 467.33971944],
       [1436.89920329],
       [2235.12936672],
       [ 228.56904751],
       [ 951.0295759 ],
       [2514.65806849],
       [ 436.67038012],
       [1527.26630174],
       [1342.54014476],
       [1584.70391747],
       [ 609.51905338],
       [ 702.07007847],
       [ 645.20265424],
       [ 745.78686972],
       [2066.08234913],
       [1078.99577626],
       [ 903.67432384],
       [1976.19133973],
       [1408.88403429],
       [ 334.31230222],
       [1786.32358272],
       [1101.22205619],
       [ 471.82048221],
       [1414.53535855],
       [1513.06996872],
       [ 766.31284335],
       [1521.54457993],
       [1768.54021338],
       [1642.880

In [99]:
sample = pd.read_csv("Data/sample_submission.csv")

In [100]:
sample.head()

,laptop_ID,Price_euros
0,539,650
1,327,650
2,563,650
3,13,650
4,935,500


In [101]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [102]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit.reshape(-1)})

In [103]:
submission.head()

,laptop_ID,Price_euros
0,539,1266.526278
1,327,1161.991321
2,563,264.011113
3,13,1941.696887
4,935,1616.026475


In [104]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [105]:
def chequeator(df_to_submit,version):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit['laptop_ID'].all() == sample['laptop_ID'].all():
                print("You're ready to submit!")
                submission.to_csv(f"submission_{version}.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [106]:
chequeator(submission,6.2)

You're ready to submit!
